# KMeans Spark

https://spark.apache.org/docs/latest/mllib-clustering.html#k-means

1. Start Spark Cluster:

Link to iPython Notebook: [Spark on Stampede](../02_hadoop_on_hpc/Stampede.ipynb)

In [2]:
from numpy import array
from math import sqrt

%run ../util/init_spark.py
%run ../env.py

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]

#sc = SparkContext("spark://129.114.82.28:7077", "Pilot-Spark KMeans")
sc = SparkContext("local[*]", "Pilot-Spark KMeans")
from pyspark.mllib.clustering import KMeans, KMeansModel

Init Spark: /usr/local/Cellar/apache-spark/1.4.1/libexec
SPARK HOME: /usr/local/Cellar/apache-spark/1.4.1/libexec


## MLLib KMeans

In [3]:
%time
# Load and parse the data
data = sc.textFile("../../sc15-examples-devel/k-means/dataset.in")
parsedData = data.map(lambda x: [float(x)])

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 256, maxIterations=3, runs=1, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
#def error(point):
#    center = clusters.centers[clusters.predict(point)]
#    return sqrt(sum([x**2 for x in (point - center)]))

#WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
#print("Within Set Sum of Squared Error = " + str(WSSSE))



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


Py4JJavaError: An error occurred while calling o19.trainKMeansModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 1 times, most recent failure: Lost task 3.0 in stage 4.0 (TID 119, localhost): ExecutorLostFailure (executor driver lost)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1273)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1264)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1263)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:47)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1263)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:730)
	at org.apache.spark.sc

In [5]:
str(clusters)

'<pyspark.mllib.clustering.KMeansModel object at 0x2b03471b3810>'

In [ ]:
# Save and load model
clusters.save(sc, "kmeans.model")
sameModel = KMeansModel.load(sc, "kmeans.model")